Connect to GA

In [ ]:
pip install google-auth google-analytics-data

In [1]:
from google.oauth2 import service_account
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import RunReportRequest

# Path to your credentials file
SERVICE_ACCOUNT_FILE = r"C:\Users\shagufta\Documents\Data Analyst\Python\Shagufta-Python\Google Analytics\Websitevala.json"

# Authenticate using the service account
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)

# Initialize the GA4 client with credentials
client = BetaAnalyticsDataClient(credentials=credentials)

# Replace with your GA4 property ID
property_id = "309712075"

# Function to fetch real-time data (e.g., active users)
def get_ga_data():
    request = RunReportRequest(
        property=f"properties/{property_id}",
        metrics=[{"name": "activeUsers"}],  # Metric to fetch
        date_ranges=[{"start_date": "2024-03-16", "end_date": "today"}]  # Date range to include
    )
    
    response = client.run_report(request)
    
    for row in response.rows:
        print(f"Active Users: {row.metric_values[0].value}")

# Call the function to get data
get_ga_data()


Active Users: 1130


Live_Data Pull from GA

In [8]:
import pandas as pd
from google.oauth2 import service_account
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import RunReportRequest, DateRange, Metric, Dimension

# Path to your service account key file
SERVICE_ACCOUNT_FILE = r"C:\Users\shagufta\Documents\Data Analyst\Python\Shagufta-Python\Google Analytics\Websitevala.json"

# Authenticate using the service account
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)

# Initialize the GA4 client with credentials
client = BetaAnalyticsDataClient(credentials=credentials)

# Replace with your GA4 property ID
property_id = "309712075"

# Function to fetch active users by country for last year and return as DataFrame
def get_active_users_by_country():
    # Define the date range (last year)
    date_range = DateRange(start_date="2024-01-01", end_date="2024-12-31")
    
    # Define metrics (active users)
    metrics = [
        Metric(name="activeUsers"),
    ]
    
    # Define dimensions (country)
    dimensions = [
        Dimension(name="country"),
    ]
    
    # Create the request
    request = RunReportRequest(
        property=f"properties/{property_id}",
        date_ranges=[date_range],  # Specify date range
        metrics=metrics,  # Specify metrics
        dimensions=dimensions  # Specify dimensions
    )

    try:
        # Fetch the report
        response = client.run_report(request)
        
        # Collect data in a list
        data = []
        
        # Check if data is returned
        if not response.rows:
            print("No data returned for the specified query.")
        else:
            # Process and collect the data
            for row in response.rows:
                country = row.dimension_values[0].value
                active_users = row.metric_values[0].value
                data.append([country, active_users])
        
        # Convert the collected data into a DataFrame
        df = pd.DataFrame(data, columns=["Country", "Active Users"])
        
        return df
    
    except Exception as e:
        print(f"Error fetching the report: {e}")
        return None

# Call the function to fetch the report and store it in a DataFrame
df_active_users_by_country = get_active_users_by_country()

# Display the DataFrame
if df_active_users_by_country is not None:
    print(df_active_users_by_country)


                 Country Active Users
0          United States          357
1                  India          286
2         United Kingdom           67
3                 Canada           58
4                Germany           42
5                  China           26
6               Pakistan           25
7              Singapore           22
8                Ireland           18
9              (not set)           12
10                Poland            6
11                 Japan            4
12           Netherlands            4
13           Philippines            4
14                Russia            4
15               Vietnam            4
16                France            3
17                Israel            3
18               Jamaica            3
19                Sweden            3
20  United Arab Emirates            3
21                 Egypt            2
22             Hong Kong            2
23             Indonesia            2
24                 Kenya            2
25          

Store in SQL

In [1]:
import pyodbc

In [4]:
import pyodbc
from google.oauth2 import service_account
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import RunReportRequest, DateRange, Metric, Dimension

# 🔹 SQL Server Connection Parameters
server = r"SP_PC\SQLEXPRESS"
database = "Website"
driver = "ODBC Driver 17 for SQL Server"
conn_string = f"DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes"

# 🔹 Google Analytics Credentials
SERVICE_ACCOUNT_FILE = r"pathname"
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)

# 🔹 Initialize GA4 Client
client = BetaAnalyticsDataClient(credentials=credentials)

# 🔹 Replace with your GA4 property ID
property_id = ""

# Function to Fetch Live Data from Google Analytics & Insert into SQL Server
def fetch_and_store_active_users():
    # Define the date range (last 30 minutes for real-time)
    date_range = DateRange(start_date="2024-01-31", end_date="today")

    # Define metrics (active users)
    metrics = [Metric(name="activeUsers")]

    # Define dimensions (country)
    dimensions = [Dimension(name="country")]

    # Create the request
    request = RunReportRequest(
        property=f"properties/{property_id}",
        date_ranges=[date_range],
        metrics=metrics,
        dimensions=dimensions
    )

    try:
        # Fetch the report
        response = client.run_report(request)

        # Check if data is returned
        if not response.rows:
            print("No data returned for the specified query.")
            return

        # Connect to SQL Server
        conn = pyodbc.connect(conn_string)
        cursor = conn.cursor()
        print("✅ Connected to SQL Server successfully!")

        # Insert data into SQL Server
        for row in response.rows:
            country = row.dimension_values[0].value
            active_users = int(row.metric_values[0].value)

            cursor.execute(
                "INSERT INTO Websitevala (activeUsers, country) VALUES (?, ?)",
                (active_users, country),
            )

        # Commit transaction
        conn.commit()
        print("✅ Live data inserted successfully!")

        # Close connection
        cursor.close()
        conn.close()

    except Exception as e:
        print(f"❌ Error: {e}")

# Call the function to fetch and store live data
fetch_and_store_active_users()


✅ Connected to SQL Server successfully!
✅ Live data inserted successfully!


In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
server = "SP_PC\SQLEXPRESS"
database = "Website"
driver = "ODBC Driver 17 for SQL Server" #Ensure you have correct driver, if not then install

#SQL Server authentication
#"Driver={SQL Server};Server=myServerName;Database=myDatabaseName;UID=myUsername;PWD=myPassword;"

#Windows authentication
conn_string = f"Driver={driver};Server={server};Database={database};Trusted_Connection=yes"

try:
    conn = pyodbc.connect(conn_string)
    cur = conn.cursor()
    print("Connected to SQL Server successfully!")
except exception as e:
    print("Error:",e)

#conn.close()
#cur.close()

Connected to SQL Server successfully!


In [11]:
import pandas as pd

In [12]:
query = "Select top 10* from Websitevala;"
df = pd.read_sql(query, conn)
df.head(10)


,activeUsers,country
0,474,United States
1,343,India
2,83,United Kingdom
3,61,Canada
4,47,Germany
5,32,China
6,27,Pakistan
7,22,Singapore
8,17,Ireland
9,14,(not set)
